In [1]:
"""A simple script to extract & interpolate CSL2 height map from JAXA:AW3D30 data files.

Author: HomeOnMars
"""

'A simple script to extract & interpolate CSL2 height map from JAXA:AW3D30 data files.\n\nAuthor: HomeOnMars\n'

# Functions

In [2]:
# importing


# my modules
import pycslhmap as pchm
#from pyhomhmap import get_CSL_height_maps


# other modules
from datetime import datetime, UTC

def now():
    return datetime.now(UTC)

# Examples

In [3]:
if __name__ == '__main__':

    verbose = True
    
    if verbose:
        # record used time
        time_start = now()
        #print(f"Start: {time_start.isoformat()}")

    
    
    # Example 1 - extrapolate data from tiff to CSL2-compatible 

    
    
    # download the relevant data from https://www.eorc.jaxa.jp/ALOS/en/dataset/aw3d_e.htm
    #    (or some other sources, I don't care)
    #    If you download from JAXA, you will need to register an account and read their terms of service
    #    after downloading, put them in the ./raw/ folder, and supply the file path (incl. file name) here
    #    they will be used to interpolate the elevations in the respective areas of the image.
    #    if you see a patch of the image is constant at minimal height-1,
    #    then you haven't downloaded & added the data of that patch. Probably.
    tiffilenames = [                # path of the raw data files, downloaded from JAXA or elsewhere
        'raw/ALPSMLC30_N063W017_DSM.tif',
        'raw/ALPSMLC30_N064W016_DSM.tif',
        'raw/ALPSMLC30_N064W017_DSM.tif',
    ]

    # the following function will do the job
    img_arr, coord = pchm.get_CSL_height_maps(
        long=-16.000, lati=+64.185, # longitude and latitude in degrees
        angle_deg=30.,              # the angle for the map to be rotated (in degrees)
        tiffilenames=tiffilenames,
        map_scales=(1.125, 1.0),    # scale factor-  e.g. map_scales=(1.5, 1.2) means
                                    #    stretching the width of the map to 1:1.5
                                    #    (i.e. mapping real world 1.5*57.344km to game 57.344km)
                                    #    while stretching the heights to 1:1.2
        out_filepath='./out/',      # output file folder path
        verbose=verbose)


    
    if verbose:
        # record used time
        time_ended = now()
        time_used  = time_ended - time_start
        print(
            #f"Ended: {time_ended.isoformat()}\n" +
            f"Time Used: {time_used}\n"
        )



	World map size (64.512 km)^2
Reading data from file raw/ALPSMLC30_N063W017_DSM.tif... Hit (  9.50%).
Reading data from file raw/ALPSMLC30_N064W016_DSM.tif... Hit ( 44.10%).
Reading data from file raw/ALPSMLC30_N064W017_DSM.tif... Hit ( 40.43%).
Total  94.03% of the map has been covered.
*** Warning: a large portion of the map (  5.97%) hasn't been covered by interpolation. Please consider download and add more map tiles data.
Smoothing... Done.
	maximum height = 1847.5051666931968
	Center point at longtitude 344.0, latitude 64.185
 	World Map longitude range from    343.088 to    344.905
 	          latitude  range from   +63.7893 to   +64.5807
	Smoothing Kernel radius 32.00 pixel (shore), 1.00 pixel (all)
Saving to ./out/worldmap_long344.000_lati+64.185_angle030.0_scale1.12+1.00.png

	Playable map size (16.128 km)^2
Using data from file raw/ALPSMLC30_N063W017_DSM.tif... Missed.
Using data from file raw/ALPSMLC30_N064W016_DSM.tif... Hit ( 50.00%).
Using data from file raw/ALPSMLC30_N

# Manuals

In [4]:
if __name__ == '__main__':
    help(pchm.get_CSL_height_maps)

Help on function get_CSL_height_maps in module pyhomhmap.import_hmap:

get_CSL_height_maps(long: float, lati: float, tiffilenames: tuple[str], cityname: str = None, angle_deg: float = 0.0, map_scales: float | tuple[float, float] = 1.0, height_scale: float = 4096.0, min_height_m: int = 100, ocean_height: int = 50, smooth_shore_rad_m: float = 448.0, smooth_rad_m: float = 14.0, interp_method: str = 'linear', opened_data: dict = {'raw/ALPSMLC30_N063W017_DSM.tif': <scipy.interpolate._rgi.RegularGridInterpolator object at 0x7f449c9d51c0>, 'raw/ALPSMLC30_N064W016_DSM.tif': <scipy.interpolate._rgi.RegularGridInterpolator object at 0x7f449e8182f0>, 'raw/ALPSMLC30_N064W017_DSM.tif': <scipy.interpolate._rgi.RegularGridInterpolator object at 0x7f449d5169c0>}, Rearth_km: float = 6378.1, out_filepath: None | str = './out/', verbose: bool = True)
    Wrapper function to extract height map from data and save them to disk.

    angle_deg: float
        how many degrees we are rotating the map counter-c